In [24]:
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

import pandas as pd
import sqlite3

In [25]:
conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext(conf=conf)

conn = sqlite3.connect("../bsg_simple.db")

In [30]:
query = "SELECT * FROM people"

# Read data into a pandas DataFrame
df_people = pd.read_sql_query(query, conn)

# Show the DataFrame
print(df_people.head())

rdd_people = sc.parallelize(df_people.values.tolist())

rdd_people.collect()

   id first_name last_name salutation call_sign  home_colony_id
0   1        Lee     Adama        Sir    Apollo             4.0
1   2    William     Adama        Sir    Husker             4.0
2   3       Kara    Thrace        Sir  Starbuck             4.0
3   4      Gaius    Baltar        Dr.      None             1.0
4   5      Laura    Roslin        Ms.      None             4.0


[[1, 'Lee', 'Adama', 'Sir', 'Apollo', 4.0],
 [2, 'William', 'Adama', 'Sir', 'Husker', 4.0],
 [3, 'Kara', 'Thrace', 'Sir', 'Starbuck', 4.0],
 [4, 'Gaius', 'Baltar', 'Dr.', None, 1.0],
 [5, 'Laura', 'Roslin', 'Ms.', None, 4.0],
 [6, 'Leoben', 'Conoy', 'Mr.', None, nan],
 [7, 'Sharon', 'Valerii', 'Sir', 'Boomer', nan],
 [8, 'Sharon', 'Agathon', 'Sir', 'Athena', nan],
 [9, 'Saul', 'Tigh', 'Sir', None, 13.0],
 [10, 'Aaron', 'Doral', 'Mr.', None, nan],
 [11, 'Billy', 'Keikeya', 'Mr.', None, 8.0],
 [12, 'Galen', 'Tyrol', 'Sir', None, 13.0],
 [13, 'Anastasia', 'Dualla', 'Sir', None, 9.0],
 [14, 'Felix', 'Gaeta', 'Sir', None, nan],
 [15, 'Cally', 'Henderson', 'Sir', None, nan],
 [16, 'Karl', 'Agathon', 'Sir', 'Helo', 4.0],
 [17, 'Samuel T.', 'Anders', 'Sir', None, 13.0]]

In [23]:
conn.close()
sc.stop()